In [1]:
import pandas as pd
from datetime import datetime
import os

In [21]:
price_usd = pd.read_csv('../input/btc_usd.csv')
trades = pd.read_csv('../temporary/trades_combined.csv')
trades

date       tid       price    amount Currency
0         1664328207  56510490    98325.52  0.001017      AED
1         1664278646  56507029    72718.80  0.233777      AED
2         1664262787  56506374   103802.28  0.000963      AED
3         1664213474  56503957    67281.57  0.460000      AED
4         1664036001  56496319    66769.49  0.030000      AED
...              ...       ...         ...       ...      ...
43024557  1546936993   2609678  5141000.51  0.007294      VES
43024558  1546937422   2609679  5169989.25  0.001934      VES
43024559  1546937551   2609680  4993996.35  0.004606      VES
43024560  1546938200   2609681  5280099.85  0.044980      VES
43024561  1546938807   2609682  5175519.79  0.003014      VES

[43024562 rows x 5 columns]

In [22]:
price_usd = pd.read_csv('../input/btc_usd.csv')
price_usd['date'] = pd.to_datetime(price_usd['Date'])
price_usd['date'] = pd.to_datetime(price_usd['date'].dt.strftime('%Y-%m-%d'))
price_usd = price_usd[['date', 'Close']]
price_usd

date         Close
0    2017-02-01    989.023010
1    2017-02-02   1011.799988
2    2017-02-03   1029.910034
3    2017-02-04   1042.900024
4    2017-02-05   1027.339966
...         ...           ...
2095 2022-10-28  20595.351563
2096 2022-10-29  20818.476563
2097 2022-10-30  20635.603516
2098 2022-10-31  20495.773438
2099 2022-11-01  20409.154297

[2100 rows x 2 columns]

In [23]:
trades['date'] = pd.to_datetime(trades['date'], unit = 's')
trades

date       tid       price    amount Currency
0        2022-09-28 01:23:27  56510490    98325.52  0.001017      AED
1        2022-09-27 11:37:26  56507029    72718.80  0.233777      AED
2        2022-09-27 07:13:07  56506374   103802.28  0.000963      AED
3        2022-09-26 17:31:14  56503957    67281.57  0.460000      AED
4        2022-09-24 16:13:21  56496319    66769.49  0.030000      AED
...                      ...       ...         ...       ...      ...
43024557 2019-01-08 08:43:13   2609678  5141000.51  0.007294      VES
43024558 2019-01-08 08:50:22   2609679  5169989.25  0.001934      VES
43024559 2019-01-08 08:52:31   2609680  4993996.35  0.004606      VES
43024560 2019-01-08 09:03:20   2609681  5280099.85  0.044980      VES
43024561 2019-01-08 09:13:27   2609682  5175519.79  0.003014      VES

[43024562 rows x 5 columns]

In [24]:
trades = trades.sort_values(by='date')
trades = trades.drop_duplicates(subset = ['tid']) #drop duplicates
trades = trades[trades['date'] > '2017-03-15'] #only use trades March 15, 2017 and beyond
trades

date       tid        price    amount Currency
10649809 2017-03-15 00:00:13   6277278     26490.07  0.075500      MXN
40515110 2017-03-15 00:00:15    100281   4126984.13  0.031500      VEF
39172389 2017-03-15 00:00:15   6277265      1292.97  0.657400      USD
7852718  2017-03-15 00:00:24   6277292      1068.00  0.110000      GBP
7852716  2017-03-15 00:00:40   6277296      1062.93  0.117600      GBP
...                      ...       ...          ...       ...      ...
3154199  2022-09-28 01:55:04  56510551  84041264.26  0.002856      COP
36360044 2022-09-28 01:55:10  56510558     18029.58  0.031892      USD
35810034 2022-09-28 01:55:10  56510556     21699.47  0.005530      USD
35810035 2022-09-28 01:57:02  56510553     20045.22  0.002494      USD
10092414 2022-09-28 01:59:11  56510550  28891713.86  0.003461      KRW

[42636062 rows x 5 columns]

In [25]:
#Add nominal amount of crypto bought, both in domestic currency and USD
trades['amount_nominal'] = trades['price']*trades['amount']
trades = pd.merge_asof(trades, price_usd, on = 'date')
trades['amount_usd'] = trades['amount']*trades['Close']
trades = trades.drop(columns = ['Close'])
trades

date       tid        price    amount Currency  \
0        2017-03-15 00:00:13   6277278     26490.07  0.075500      MXN   
1        2017-03-15 00:00:15    100281   4126984.13  0.031500      VEF   
2        2017-03-15 00:00:15   6277265      1292.97  0.657400      USD   
3        2017-03-15 00:00:24   6277292      1068.00  0.110000      GBP   
4        2017-03-15 00:00:40   6277296      1062.93  0.117600      GBP   
...                      ...       ...          ...       ...      ...   
42636057 2022-09-28 01:55:04  56510551  84041264.26  0.002856      COP   
42636058 2022-09-28 01:55:10  56510558     18029.58  0.031892      USD   
42636059 2022-09-28 01:55:10  56510556     21699.47  0.005530      USD   
42636060 2022-09-28 01:57:02  56510553     20045.22  0.002494      USD   
42636061 2022-09-28 01:59:11  56510550  28891713.86  0.003461      KRW   

          amount_nominal         Close  amount_usd  
0            2000.000285   1249.609985   94.345554  
1          130000.000095   1249.609985   39.362715  
2             849.998478   1249.609985  821.493604  
3             117.480000   1249.609985  137.457098  
4             125.000568   1249.609985  146.954134  
...                  ...           ...         ...  
42636057   239999.999998  19426.720703   55.477663  
42636058      575.000087  19426.720703  619.557754  
42636059      120.000022  19426.720703  107.431514  
42636060       49.999995  19426.720703   48.457235  
42636061   100000.000012  19426.720703   67.239766  

[42636062 rows x 8 columns]

In [27]:
trades

date       tid        price    amount Currency  \
0        2017-03-15 00:00:13   6277278     26490.07  0.075500      MXN   
1        2017-03-15 00:00:15    100281   4126984.13  0.031500      VEF   
2        2017-03-15 00:00:15   6277265      1292.97  0.657400      USD   
3        2017-03-15 00:00:24   6277292      1068.00  0.110000      GBP   
4        2017-03-15 00:00:40   6277296      1062.93  0.117600      GBP   
...                      ...       ...          ...       ...      ...   
42636057 2022-09-28 01:55:04  56510551  84041264.26  0.002856      COP   
42636058 2022-09-28 01:55:10  56510558     18029.58  0.031892      USD   
42636059 2022-09-28 01:55:10  56510556     21699.47  0.005530      USD   
42636060 2022-09-28 01:57:02  56510553     20045.22  0.002494      USD   
42636061 2022-09-28 01:59:11  56510550  28891713.86  0.003461      KRW   

          amount_nominal  amount_usd  
0            2000.000285   94.345554  
1          130000.000095   39.362715  
2             849.998478  821.493604  
3             117.480000  137.457098  
4             125.000568  146.954134  
...                  ...         ...  
42636057   239999.999998   55.477663  
42636058      575.000087  619.557754  
42636059      120.000022  107.431514  
42636060       49.999995   48.457235  
42636061   100000.000012   67.239766  

[42636062 rows x 7 columns]

## Prepare data for algorithm

In [28]:
# trades.set_index('date', inplace=True)
trades['counter'] = 1
trades

date       tid        price    amount Currency  \
0        2017-03-15 00:00:13   6277278     26490.07  0.075500      MXN   
1        2017-03-15 00:00:15    100281   4126984.13  0.031500      VEF   
2        2017-03-15 00:00:15   6277265      1292.97  0.657400      USD   
3        2017-03-15 00:00:24   6277292      1068.00  0.110000      GBP   
4        2017-03-15 00:00:40   6277296      1062.93  0.117600      GBP   
...                      ...       ...          ...       ...      ...   
42636057 2022-09-28 01:55:04  56510551  84041264.26  0.002856      COP   
42636058 2022-09-28 01:55:10  56510558     18029.58  0.031892      USD   
42636059 2022-09-28 01:55:10  56510556     21699.47  0.005530      USD   
42636060 2022-09-28 01:57:02  56510553     20045.22  0.002494      USD   
42636061 2022-09-28 01:59:11  56510550  28891713.86  0.003461      KRW   

          amount_nominal  amount_usd  counter  
0            2000.000285   94.345554        1  
1          130000.000095   39.362715        1  
2             849.998478  821.493604        1  
3             117.480000  137.457098        1  
4             125.000568  146.954134        1  
...                  ...         ...      ...  
42636057   239999.999998   55.477663        1  
42636058      575.000087  619.557754        1  
42636059      120.000022  107.431514        1  
42636060       49.999995   48.457235        1  
42636061   100000.000012   67.239766        1  

[42636062 rows x 8 columns]

In [29]:
path = '../temporary/trades_cleaned.csv'
trades.to_csv(path, index=False)